In [1]:
# ==============================================
# COMMON IMPORTS (Run this cell first)
# ==============================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
import seaborn as sns
import os
import zipfile
import requests
import io

# Set random seeds for reproducibility (optional)
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)

/Users/aditya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


TensorFlow Version: 2.19.0
Keras Version: 3.9.2


In [ ]:
# ==============================================
# Assignment 5: Image Classification CNN (CIFAR-10)
# ==============================================
print("\n--- Assignment 5: Image Classification CNN (CIFAR-10) ---")

# --- 1. Load and Preprocess the CIFAR-10 Dataset ---
print("Loading CIFAR-10 dataset...")
(x_train_cifar, y_train_cifar), (x_test_cifar, y_test_cifar) = keras.datasets.cifar10.load_data()

# Preprocess Data: Normalize pixel values, one-hot encode labels
num_classes_cifar = 10
input_shape_cifar = x_train_cifar.shape[1:] # Should be (32, 32, 3)

x_train_cifar = x_train_cifar.astype('float32') / 255.0
x_test_cifar = x_test_cifar.astype('float32') / 255.0

y_train_cifar = keras.utils.to_categorical(y_train_cifar, num_classes_cifar)
y_test_cifar = keras.utils.to_categorical(y_test_cifar, num_classes_cifar)

cifar_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                'dog', 'frog', 'horse', 'ship', 'truck']

print(f"x_train shape: {x_train_cifar.shape}")
print(f"y_train shape: {y_train_cifar.shape}")
print(f"Input shape: {input_shape_cifar}")

# --- 2. Define the CNN Model (Slightly deeper for CIFAR-10) ---
print("Building CIFAR-10 CNN model...")
model_cifar_cnn = keras.Sequential(
    [
        keras.Input(shape=input_shape_cifar),
        layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation="relu"),
        layers.BatchNormalization(), # Helps stabilize training
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),

        layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes_cifar, activation="softmax"),
    ],
    name="cifar10_cnn"
)
model_cifar_cnn.summary()

# --- 3. Compile the Model ---
print("Compiling CIFAR-10 CNN model...")
# Using a slightly lower learning rate can sometimes help for CIFAR-10
optimizer_cifar = keras.optimizers.Adam(learning_rate=0.001)
model_cifar_cnn.compile(loss="categorical_crossentropy", optimizer=optimizer_cifar, metrics=["accuracy"])

# --- 4. Train the Model ---
print("Training CIFAR-10 CNN model...")
batch_size_cifar = 64 # Smaller batch size often used for CIFAR-10
epochs_cifar = 10 # Reduced for quick demo; needs more epochs (e.g., 25-50+) for good accuracy
history_cifar_cnn = model_cifar_cnn.fit(x_train_cifar, y_train_cifar,
                                        batch_size=batch_size_cifar,
                                        epochs=epochs_cifar,
                                        validation_split=0.1,
                                        verbose=1)

# --- 5. Evaluate the Model ---
print("Evaluating CIFAR-10 CNN model...")
score_cifar_cnn = model_cifar_cnn.evaluate(x_test_cifar, y_test_cifar, verbose=0)
print(f"Test loss: {score_cifar_cnn[0]:.4f}")
print(f"Test accuracy: {score_cifar_cnn[1]:.4f}") # Accuracy will be lower than MNIST with few epochs

# --- Plot training history (Optional) ---
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history_cifar_cnn.history['accuracy'], label='Train Accuracy')
plt.plot(history_cifar_cnn.history['val_accuracy'], label='Validation Accuracy')
plt.title('Assignment 5: CIFAR-10 CNN Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_cifar_cnn.history['loss'], label='Train Loss')
plt.plot(history_cifar_cnn.history['val_loss'], label='Validation Loss')
plt.title('Assignment 5: CIFAR-10 CNN Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

# --- Visualize Predictions (Optional) ---
predictions_cifar = model_cifar_cnn.predict(x_test_cifar[:10])
predicted_labels_cifar = np.argmax(predictions_cifar, axis=1)
true_labels_cifar = np.argmax(y_test_cifar[:10], axis=1)

plt.figure(figsize=(15, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_test_cifar[i])
    pred_label = cifar_labels[predicted_labels_cifar[i]]
    true_label = cifar_labels[true_labels_cifar[i]]
    plt.title(f"Pred: {pred_label}\nTrue: {true_label}",
              color=("green" if pred_label == true_label else "red"))
    plt.axis('off')
plt.suptitle("Assignment 5: CIFAR-10 Sample Predictions")
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
plt.show()


--- Assignment 5: Image Classification CNN (CIFAR-10) ---
Loading CIFAR-10 dataset...
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 194s 1us/step 
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 10)
Input shape: (32, 32, 3)
Building CIFAR-10 CNN model...


Model: "cifar10_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 30, 30, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 15, 15, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 15, 15, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,253,674 (4.78 MB)

 Trainable params: 1,252,266 (4.78 MB)

 Non-trainable params: 1,408 (5.50 KB)

Compiling CIFAR-10 CNN model...
Training CIFAR-10 CNN model...
Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 106s 146ms/step - accuracy: 0.3770 - loss: 2.0231 - val_accuracy: 0.5988 - val_loss: 1.1373
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 112s 159ms/step - accuracy: 0.6072 - loss: 1.1180 - val_accuracy: 0.6700 - val_loss: 0.9412
Epoch 3/10
183/704 ━━━━━━━━━━━━━━━━━━━━ 1:58 228ms/step - accuracy: 0.6547 - loss: 0.9701